In [1]:
from PIL import Image
from tqdm import tqdm
import hashlib
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
import Augmentor

from sklearn.model_selection import StratifiedKFold, KFold

from keras.applications.vgg19 import VGG19, preprocess_input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout

Using TensorFlow backend.


In [2]:
AUGMENT_FACTOR = 5
IMG_SHAPE = (224, 224)

In [3]:
path = Path("dataset_updated")

In [4]:
img_list = []
for img in path.glob('*/*/*'):
    try:
        Image.open(img)
    except:
        continue
    cat = img.parts[-2]
    pathstr = str(img.absolute())
    img_id = hashlib.md5(pathstr.encode('utf-8')).hexdigest()
    img_list.append((img_id, pathstr, cat))
img_df = pd.DataFrame(img_list, columns=['id', 'path', 'category'])
img_df.index = img_df.id
img_df.drop('id', axis=1, inplace=True)
img_df.category = img_df.category.astype('category')
img_df.head()

/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


,path,category
id,,
96cb78a545ae9bff37048375762a43c4,/home/asm/dev/budokai/dataset_updated/validati...,iconography
c8c4ef86d86573cacb5991e362a4ac2d,/home/asm/dev/budokai/dataset_updated/validati...,iconography
1eac68a564617590992bae5e51fd4d8e,/home/asm/dev/budokai/dataset_updated/validati...,iconography
0ddef4a9979cb84a1855c6e5133bd88c,/home/asm/dev/budokai/dataset_updated/validati...,iconography
08d8de80b60f2373689afa401ad4c547,/home/asm/dev/budokai/dataset_updated/validati...,iconography


In [ ]:
def apply_pipeline(df, sample_size):
    
    p = Augmentor.DataFramePipeline(df, 'path', 'category')
    p.flip_random(0.5)
    p.gaussian_distortion(0.1, 4, 4, 3, 'bell', 'in')
    p.random_distortion(0.3, 4, 4, 8)
    p.shear(0.1, 10, 10)
    p.skew(0.1)
    p.rotate(0.3, 20, 20)
    p.rotate_random_90(0.1)
    p.resize(1, *IMG_SHAPE)
    
    xlen = len(p.augmentor_images)
    if sample_size > xlen:
        samples = np.concatenate((np.arange(xlen), np.random.choice(xlen, size=sample_size-xlen)), axis=0)
    else:
        samples = np.random.choice(xlen, size=sample_size, replace=False)
        
    with tqdm(total=len(samples), desc="Generating samples", unit="samples") as pbar:
        
        def process(img):
            aug_img = np.array(p._execute(img, save_to_disk=False))
            if aug_img.ndim < 3:
                aug_img = np.stack([aug_img]*3, -1)
            if aug_img.shape[-1] > 3:
                aug_img = aug_img[..., :3]
            assert aug_img.shape == (*IMG_SHAPE, 3), img.image_path
            pbar.update(1)
            return aug_img, img.categorical_label
        
        X, y = zip(*map(process, np.array(p.augmentor_images)[samples]))
    
    return np.array(X), np.array(y), df.index[samples]

In [ ]:
# X, y, idx = apply_pipeline(img_df, AUGMENT_FACTOR * len(img_df))
data = np.load('full_data.npz')
X, y = data['X'], data['y']
X.shape

(42885, 224, 224, 3)

In [ ]:
# np.savez('full_data.npz', X=X, y=y, idx=idx.values)

In [ ]:
N = 16
b = np.ceil(len(X)/N).astype(int)
for i in range(N):
    X[i*b:(i+1)*b] = preprocess_input(X[i*b:(i+1)*b])

In [ ]:
base_model = VGG19(weights='imagenet')

x = base_model.get_layer('block5_pool').output
x = GlobalAveragePooling2D(name='global_avg')(x)
x = Dense(1024, activation='relu', name='features')(x)
# x = Dropout(0.5, name='dropout')(x)
y_out = Dense(5, activation='softmax', name='prediction')(x)

model = Model(inputs=base_model.input, outputs=y_out)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
model.fit(X, y, batch_size=8, epochs=5)

Epoch 1/10
42885/42885 [==============================] - 418s 10ms/step - loss: 4.1273
Epoch 2/10
13832/42885 [========>.....................] - ETA: 4:40 - loss: 3.4614